# 포스터 학습
##### 포스터와 라벨링 데이터를 학습하여 텍스트 위치를 예측하는 AI
##### 데이터가 적어 랜덤 포레스트 회기로 학습함. 추후 데이터가 커지면 transformer로 업그레이드 하는게 좋을듯

In [38]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib

In [12]:
# 데이터 불러오기
labeled_data = '../poster_labeling/labeled_posters.csv'
data = pd.read_csv(labeled_data)

# 데이터 준비
character_data = data[data['Object_Type'] == 'Character']
text_data = data[data['Object_Type'] == 'Text']

# Character와 Text 병합
merged_data = pd.merge(
    character_data,
    text_data,
    on='Poster_File',
    suffixes=('_char', '_text')
)

# 주요 컬럼 추출
prepared_data = merged_data[[
    'Poster_File',
    'X1_char', 'Y1_char', 'X2_char', 'Y2_char',
    'X1_text', 'Y1_text', 'X2_text', 'Y2_text'
]]

# 입력과 출력 데이터 준비
X = prepared_data[['X1_char', 'Y1_char', 'X2_char', 'Y2_char']].values  # Character 좌표
y = prepared_data[['X1_text', 'Y1_text', 'X2_text', 'Y2_text']].values  # Text 좌표

# 데이터 분할 (90% 학습, 10% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [35]:
# 랜덤 포레스트 회귀 모델 초기화 및 학습
rf_model = RandomForestRegressor(
    n_estimators=70,       # 트리 개수
    max_depth=10,          # 트리의 최대 깊이
    min_samples_split=5,   # 노드 분할 최소 샘플
    min_samples_leaf=2,    # 리프 노드 최소 샘플
    random_state=42        # 재현성을 위한 난수 고정
)
rf_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, min_samples_leaf=2, min_samples_split=5,
                      n_estimators=70, random_state=42)

In [36]:
# 예측 및 평가
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"RMSE: {rmse:.2f}")
print(f"RMSE/X: {rmse/360:.2f}")
print(f"RMSE/Y: {rmse/640:.2f}")

RMSE: 105.76
RMSE/X: 0.29
RMSE/Y: 0.17


In [30]:
# 하이퍼파라미터 최적화 코드 
# 파라미터 최적화 할때만 실행하면 됨, 데이터가 작아 그리드 서치 이용, 나중에 데이터가 커지면 랜덤서치로 변경
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# 하이퍼파라미터 그리드 정의
param_grid = {
    'n_estimators': [50, 70, 100],
    'max_depth': [10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}

# 랜덤 포레스트 모델 생성
rf = RandomForestRegressor(random_state=42)

# GridSearchCV 설정
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=3,  # 교차 검증 fold 수
    scoring='neg_mean_squared_error',  # MSE 기준으로 평가
    n_jobs=-1  # 병렬 처리
)

# 학습 데이터로 그리드 서치 실행
grid_search.fit(X_train, y_train)

# 최적 하이퍼파라미터와 성능 출력
best_params = grid_search.best_params_
best_score = -grid_search.best_score_

print(f"최적 하이퍼파라미터: {best_params}")
print(f"최적 MSE: {best_score:.2f}")


최적 하이퍼파라미터: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 70}
최적 MSE: 13186.15


In [40]:
# 학습된 모델 저장 코드
# 모델 파일 생성할때만 실행해도 됨
joblib.dump(rf_model, 'text_locator.pkl')
print(f"모델이 파일로 저장되었습니다.")

모델이 파일로 저장되었습니다.
